In [1]:
import pandas as pd
import re
import emoji

data = pd.read_csv("reddit.csv")

/Users/pgalli/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
data.sample(5)

,text,label
19473,Anyone here also do not off themselves already...,SuicideWatch
15253,Why do they always doubt and never fully under...,depression
11551,I cannot fucking do this anymore. I do not kno...,SuicideWatch
14421,She died a few months an I cannot stop thinkin...,SuicideWatch
19603,Miss the ease of things I hurt so much and fea...,depression


In [3]:
df = data.copy()

In [4]:
from wordsegment import load, segment
load()

def process_hashtags(tag):
    tag = tag.lstrip('#')
    if not tag:
        return ""
    
    words = segment(tag.lower())
    return ' '.join(words)


def clean_posts(text):
    '''This function is used to clean all the text in the post_text column in the dataframe.
    It elimates emojis, urls, @/mentions, hashtags, numbers, special characters, multiple spaces,
    and converts all letters to lower case.'''
    # Replace emoji's
    text = emoji.replace_emoji(text, replace="")
    
    # Remove any urls
    text = re.sub(r"https/S+|www\S+", "", text)
    
    # Remove @/mentions
    text = re.sub(r"@\w+", "", text)
    
    # Remove hashtags
    text = re.sub(r"#(\w+)", lambda m: process_hashtags(m.group(0)), text)

    
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    
    # Remove special characters/extra puncuation
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    # Lower case
    text = text.lower()
    
    # Eliminate multiple spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [5]:
df["clean_post"] = df["text"].apply(clean_posts)

In [7]:
df.head()

,text,label,clean_post
0,I recently went through a breakup and she said...,depression,i recently went through a breakup and she said...
1,"I do not know how to navigate these feelings, ...",depression,i do not know how to navigate these feelings n...
2,"So I have been with my bf for 5 months , and h...",depression,so i have been with my bf for months and he al...
3,I am so exhausted of this. Just when I think I...,SuicideWatch,i am so exhausted of this just when i think i ...
4,I have been severly bullied since i was 5 till...,depression,i have been severly bullied since i was till t...


In [8]:
df = df[['clean_post']]

In [9]:
df.head(5)

,clean_post
0,i recently went through a breakup and she said...
1,i do not know how to navigate these feelings n...
2,so i have been with my bf for months and he al...
3,i am so exhausted of this just when i think i ...
4,i have been severly bullied since i was till t...


In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['clean_post'] = df['clean_post'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df.head()

,clean_post
0,recently went breakup said still wants friends...
1,know navigate feelings new feeling stretch und...
2,bf months already told depressed week nothing ...
3,exhausted think finally rest think maybe thing...
4,severly bullied since till resulted depressed ...


In [11]:
df.to_csv("reddit_clean_data.csv", index=False)